In [ ]:
import pandas as pd
import numpy as np

# === Load data ===
details_df = pd.read_csv("dmv_offices_details.csv")  # Contains office names and zip codes
travel_matrix_df = pd.read_csv("dmv_travel_matrix.csv", index_col=0)  # d̃(x,y) travel-time matrix
population_df = pd.read_csv("dmv_offices_with_population.csv")  # Zip-level population

# === Prepare the population lookup ===
zip_pop_dict = dict(zip(population_df['zip_code'].astype(str), population_df['population']))

# === Define the symmetric distance matrix function ===
def calc_dmv_d_matrix(dtilde_df, dmv_details_df, zip_pop_dict):
    office_names = dmv_details_df['office_name'].tolist()
    N = len(office_names)
    d_matrix = np.zeros((N, N))
    
    # Ensure the travel matrix is aligned with office names
    dtilde = dtilde_df.loc[office_names, office_names].values
    zip_codes = dmv_details_df['zip_code'].astype(str).tolist()
    
    for i in range(N):
        for j in range(N):
            if i != j:
                zipi = zip_codes[i]
                zipj = zip_codes[j]
                popi = zip_pop_dict.get(zipi, 0)
                popj = zip_pop_dict.get(zipj, 0)
                if popi == 0 and popj == 0:
                    d_matrix[i, j] = 0
                elif popi == 0:
                    d_matrix[i, j] = dtilde[j, i]
                elif popj == 0:
                    d_matrix[i, j] = dtilde[i, j]
                else:
                    d_matrix[i, j] = (popj * dtilde[i, j] + popi * dtilde[j, i]) / (popi + popj)
    return d_matrix
